In [9]:

from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import RDF
from dateutil.parser import parse
import yaml
from datetime import datetime
import pandas as pd
import csv


#instantiate namespace
schema = Namespace("http://schema.org/")
dcat = Namespace("http://www.w3.org/ns/dcat#")
dct = Namespace("http://purl.org/dc/terms/")
void= Namespace("http://rdfs.org/ns/void#")
cube= Namespace("https://cube.link/")
shacl= Namespace("http://www.w3.org/ns/shacl#")
xsd= Namespace("http://www.w3.org/2001/XMLSchema#")
qudt= Namespace("http://qudt.org/schema/qudt/")




#parse input
with open("C:\\Users\\claudio\\Documents\\Projekte\\python lottogewinne\\input_Form.yml", "rt", encoding='utf8') as yml_input:
    input_data = yaml.load(yml_input, yaml.Loader)
    
try:
    with open("C:\\Users\\claudio\\Documents\\Projekte\\python lottogewinne\\ODS_Form.yml", "rt", encoding='utf8') as ods_input:
        ODS_data = yaml.load(ods_input, yaml.Loader)
    if ODS_data.get("identifier"):
        ODS_flag = True
    else:
        ODS_flag = False
        print("No opendata.swiss metadata was provided")
except(FileNotFoundError):
    print("No opendata.swiss metadata form was provided")



#instantiate graph
g = Graph()

#could add the option to read an existing graph (for example a generated cube constraint and observation set)

#set dataset URI to attach metadata to
dataset_URL = input_data.get("dataset-URI")
#if dataset_URL[-1] != "/":
#    dataset_URL = dataset_URL+"/" # To do check, if this is correct (probably not) if reversed cube-constrain and cube observation is broken (under Dimension)
dataset = URIRef(dataset_URL)
g.add((dataset, RDF.type, schema.Dataset))
#if ODS_flag:
g.add((dataset, RDF.type, dcat.Dataset))
g.add((dataset, RDF.type, void.Dataset))
g.add((dataset, RDF.type, cube.Cube))

#for each metadata point the following steps need to be checked through
# 1. get from dict
# 2. check rough conformance of datatype if possible
# 3. assign URIRef or Literal
# 4. write triples

#name
name_DE = input_data.get("name_DE")
if name_DE:
    de_name = Literal(name_DE.strip(), lang="de")
    g.add((dataset, schema.name, de_name))
    if ODS_flag:
        g.add((dataset, dct.title, de_name))
else:
    print("Missing German dataset name")
name_FR = input_data.get("name_FR")
if name_FR:
    fr_name = Literal(name_FR.strip(), lang="fr")
    g.add((dataset, schema.name, fr_name))
    if ODS_flag:
        g.add((dataset, dct.title, fr_name))
else:
    print("Missing French dataset name")
name_IT = input_data.get("name_IT")
if name_IT:
    it_name = Literal(name_IT.strip(), lang="it")
    g.add((dataset, schema.name, it_name))
    if ODS_flag:
        g.add((dataset, dct.title, it_name))
else:
    print("Missing Italian dataset name")
name_EN = input_data.get("name_EN")
if name_EN:
    en_name = Literal(name_EN.strip(), lang="en")
    g.add((dataset, schema.name, en_name))
    if ODS_flag:
        g.add((dataset, dct.title, en_name))
else:
    print("Missing English dataset name")
description_DE = input_data.get("description_DE")
if description_DE:
    de_description = Literal(description_DE.strip(), lang="de")
    g.add((dataset, schema.description, de_description))
    if ODS_flag:
        g.add((dataset, dct.description, de_description))
else:
    print("Missing German dataset description")
description_FR = input_data.get("description_FR")
if description_FR:
    fr_description = Literal(description_FR.strip(), lang="fr")
    g.add((dataset, schema.description, fr_description))
    if ODS_flag:
        g.add((dataset, dct.description, fr_description))
else:
    print("Missing French dataset description")
description_IT = input_data.get("description_IT")
if description_IT:
    it_description = Literal(description_IT.strip(), lang="it")
    g.add((dataset, schema.description, it_description))
    if ODS_flag:
        g.add((dataset, dct.description, it_description))
else:
    print("Missing Italian dataset description")
description_EN = input_data.get("description_EN")
if description_EN:
    en_description = Literal(description_EN.strip(), lang="en")
    g.add((dataset, schema.description, en_description))
    if ODS_flag:
        g.add((dataset, dct.description, en_description))
else:
    print("Missing English dataset description")

#contact point
contact_name = input_data.get("contact-point name")
contact_mail = input_data.get("contact-point email")
if contact_name and contact_mail:
    if "@" in contact_mail:
        contact_point = BNode()
        con_name = Literal(contact_name)
        con_mail = Literal(contact_mail)
        g.add((dataset, schema.contactPoint, contact_point))
        g.add((contact_point, schema.name, con_name))
        g.add((contact_point, schema.email, con_mail))
        if ODS_flag:
            ODS_contact = BNode()
            g.add((dataset, dcat.contactPoint, ODS_contact))
            g.add((ODS_contact, RDF.type, URIRef("http://www.w3.org/2006/vcard/ns#Organization")))
            g.add((ODS_contact, URIRef("http://www.w3.org/2006/vcard/ns#fn"), con_name))
            g.add((ODS_contact, URIRef("http://www.w3.org/2006/vcard/ns#hasEmail"), con_mail))
    else:
        print("Contact point email address is not in correct format")
elif contact_name:
    print("Contact Point email address is missing")
else:
    print("Contact Point name is missing")

#dates: to be tested using datetime
date_format = "%Y-%m-%d"

#creation date
creation_date = input_data.get("source creation-date")
if creation_date:
    try: 
        datetime.strptime(str(creation_date), date_format)
        cre_date = Literal(creation_date, datatype=URIRef('http://www.w3.org/2001/XMLSchema#date'))
        g.add((dataset, schema.dateCreated, cre_date))
    except(ValueError):
        print("creation date is not formatted as xsd:date")
else:
    print("Missing source creation date")

#modification date
modification_date = input_data.get("source modification-date")
if modification_date:
    try: 
        parse(str(modification_date))
        mod_date = Literal(modification_date, datatype=URIRef('http://www.w3.org/2001/XMLSchema#dateTime'))
        g.add((dataset, schema.dateModified, mod_date))
        if ODS_flag:
            g.add((dataset, dct.modified, mod_date))
    except(ValueError):
        print("modification date is not formatted as xsd:dateTime")
else:
    print("Missing source modification date")

#publication date
publication_date = input_data.get("dataset publication-date")
if publication_date:
    try: 
        datetime.strptime(str(publication_date), date_format)
        pub_date = Literal(publication_date, datatype=URIRef('http://www.w3.org/2001/XMLSchema#date'))
        g.add((dataset, schema.datePublished, pub_date))
        if ODS_flag:
            issue_date = parse(str(publication_date))
            g.add((dataset, dct.issued, Literal(issue_date, datatype=URIRef('http://www.w3.org/2001/XMLSchema#date'))))
    except(ValueError):
        print("publication date is not formatted as xsd:date")
else:
    print("Missing dataset publication date")
    
#work example application
work_example_app = input_data.get("work example application").split(";")
for app in work_example_app:
    if app.strip() == "visualize":
        visualize_link = URIRef("https://ld.admin.ch/application/visualize")
        g.add((dataset, schema.workExample, visualize_link))
        #could also be made redundant if automatically added when there is a visualize work example. Left for now in case there are datasets that are on visualize without providing a work example URL
    elif app.strip() == "opendata.swiss":
        ODS_link = URIRef("https://ld.admin.ch/application/opendataswiss")
        g.add((dataset, schema.workExample, ODS_link))
        #possibly this metadata can be used to determine whether ODS metadata should be added or there could be a separate field in the YAML form for whether ODS publication is desired, which automatically adds this triple as well
    else:
        if app.strip():
            other_app_URI = URIRef("https://ld.admin.ch/application/"+app.strip())
            g.add((dataset, schema.workExample, other_app_URI))
        else:
            pass
        #work example application YAML form field is still required even if the main two applications would be automatically added elsewhere in case of other custom applications

#work example visualize
visualize_URL = input_data.get("work example visualize")
if visualize_URL:
    if "https://visualize.admin.ch/" in visualize_URL:
        visualize_example = Literal(visualize_URL)
        visualize_BN = BNode()
        g.add((dataset, schema.workExample, visualize_BN))
        g.add((visualize_BN, schema.url, visualize_example))
        g.add((visualize_BN, schema.name, Literal("visualize.admin.ch", lang="de")))
        g.add((visualize_BN, schema.name, Literal("visualize.admin.ch", lang="fr")))
        g.add((visualize_BN, schema.name, Literal("visualize.admin.ch", lang="it")))
        g.add((visualize_BN, schema.name, Literal("visualize.admin.ch", lang="en")))
        #This is how Cube Creator formats it, but is it really necessary to add 4 triples with different language tags for the same string?
        g.add((visualize_BN, schema.encodingFormat, Literal("text/html", datatype=URIRef('http://www.w3.org/2001/XMLSchema#string'))))
        g.add((visualize_BN, RDF.type, schema.CreativeWork))
        if visualize_link:
            pass
        else:
            visualize_link = URIRef("https://ld.admin.ch/application/visualize")
            g.add((dataset, schema.workExample, visualize_link))
    else:
        print("Visualize work example link is not correctly formatted")
else:
    print("Visualize work example link is missing")
    
#work example SPARQL endpoint
example_sparql = input_data.get("work example SPARQLendpoint")
if example_sparql:
    if "https://lindas.admin.ch/sparql/" in example_sparql:
        sparql_work = Literal(example_sparql)
        sparql_BN = BNode()
        g.add((dataset, schema.workExample, sparql_BN))
        g.add((sparql_BN, schema.url, sparql_work))
        g.add((sparql_BN, schema.name, Literal("SPARQL Endpoint mit Vorauswahl des Graph", lang="de")))
        g.add((sparql_BN, schema.name, Literal("SPARQL Endpoint avec présélection du graphe", lang="fr")))
        g.add((sparql_BN, schema.name, Literal("SPARQL Endpoint con preselezione del grafo", lang="it")))
        g.add((sparql_BN, schema.name, Literal("SPARQL Endpoint with graph preselection", lang="en")))
        g.add((sparql_BN, schema.encodingFormat, Literal("application/sparql-query", datatype=URIRef('http://www.w3.org/2001/XMLSchema#string'))))
        g.add((sparql_BN, RDF.type, schema.CreativeWork))
    else:
        print("SPARQL work example link is not correctly formatted")
else:
    print("SPARQL work example link is missing")

#SPARQL endpoint
sparql_url = input_data.get("SPARQL endpoint")
if sparql_url:
    sparql_end = Literal(sparql_url)
    end_prop = URIRef("http://rdfs.org/ns/void#sparqlEndpoint")
    g.add((dataset, end_prop, sparql_end))
else:
    print("SPARQL endpoint is missing")
    
#optional metadata
creator = input_data.get("dataset creator") 
if creator:
    if "http" in creator:
        creator_URI = URIRef(creator)
        g.add((dataset, schema.creator, creator_URI))
    else:
        print("dataset creator not formatted correctly")
else:
    print("Optional: no dataset creator provided")
contributor = input_data.get("dataset contributor")
if contributor:
    if "http" in contributor:
        contributor_URI = URIRef(contributor)
        g.add((dataset, schema.contributor, contributor_URI))
    else:
        print("dataset contributor not formatted correctly")
else:
    print("Optional: no dataset contributor provided") 
publisher = input_data.get("dataset publisher")
if publisher:
    if "http" in publisher:
        publisher_URI = URIRef(publisher)
        g.add((dataset, schema.publisher, publisher_URI))
    else:
        print("dataset publisher not formatted correctly")
else:
    print("Optional: no dataset publisher provided") 

next_modification = input_data.get("next modification date") 
if next_modification:
    try: 
        datetime.strptime(str(next_modification), date_format)
        next_mod = Literal(next_modification, datatype=URIRef('http://www.w3.org/2001/XMLSchema#date'))
        g.add((dataset, URIRef("https://schema.ld.admin.ch/datasetNextDateModified"), next_mod))
    except(ValueError):
        print("next modification date is not formatted as xsd:dateTime")
    
else:
    print("Optional: no next modification date provided")

example_resource = input_data.get("example resource")
if example_resource:
    data_URI = "/".join(dataset_URL.split("/")[:3])
    if data_URI in example_resource:
        g.add((dataset, URIRef("https://schema.ld.admin.ch/exampleResource"), URIRef(example_resource)))
else:
    print("Optional: no example resource provided")

work_status = input_data.get("creative work status")
if work_status:
    try: 
        g.add((dataset, schema.creativeWorkStatus)), URIRef(work_status)
    except(ValueError):
        print("is it a draft or published? creativeworkstatus wrong")  
   
    
    
# ODS metadata
if ODS_flag:
    ID = ODS_data.get("identifier")
    if "@" in ID:
        g.add((dataset, dct.identifier, Literal(ID)))
    else: print("opendata.swiss identifier not formatted correctly")
   
    creator_ODS = ODS_data.get("creator")
    if creator_ODS:
        if "https://register.ld.admin.ch/opendataswiss/org" in creator_ODS:
            g.add((dataset, dct.creator, URIRef(creator_ODS)))
            if not creator:
                g.add((dataset, schema.creator, URIRef(creator_ODS)))
            else: pass
        else: print("opendata.swiss creator was not formatted correctly")
    else: print("opendata.swiss creator missing")
    publisher_ODS = ODS_data.get("publisher")
    if publisher_ODS:
        g.add((dataset, dct.publisher, Literal(publisher_ODS)))
    else: print("opendata.swiss publisher missing")
    license_ODS = ODS_data.get("license")
    #this script follows the current implementation of "dct:license" on LINDAS rather than the ideal description in the DCAT-AP handbook
    if license_ODS:
        if "https://" and "fedlex" and ".admin.ch" in license_ODS:
            g.add((dataset, dct.rights, URIRef(license_ODS)))
        else: print("opendata.swiss license not formatted correctly")
    else: print("opendata.swiss license missing")
    
    rights = ODS_data.get("rights")
    #this script follows the current implementation of "dct:rights" on LINDAS rather than the ideal description in the DCAT-AP handbook
    if rights:
        if "https://ld.admin.ch/vocabulary/TermsOfUse" in rights:
            g.add((dataset, dct.rights, URIRef(rights)))
        else: print("opendata.swiss rights not formatted correctly")
    else: print("opendata.swiss rights missing")
    
    accrual = ODS_data.get("accrual periodicity")
    if accrual:
        if "http://publications.europa.eu/resource/authority/frequency" in accrual:
            g.add((dataset, dct.accrualPeriodicity, Literal(accrual)))
        else: print("opendata.swiss accrual periodicity not formatted correctly")
    else: print("opendata.swiss accrual periodicity missing")
    
    themes = ODS_data.get("themes").split(";")
    if themes[0]:
        for theme in themes:
            if not theme:
                pass
            elif "https://register.ld.admin.ch/opendataswiss/category/" in theme.strip():
                    g.add((dataset, dcat.theme, URIRef(theme.strip())))
            else: print("opendata.swiss theme was not formatted correctly")
    else: print("opendata.swiss theme was not provided")
    keys_DE = ODS_data.get("keywords_DE").split(";")
    if keys_DE[0]:
        for key_DE in keys_DE:
            if key_DE.strip():
                g.add((dataset, dcat.keyword, Literal(key_DE.strip(), lang="de")))
            else: pass
    else: print("opendata.swiss german keywords were not provided")      
    keys_FR = ODS_data.get("keywords_FR").split(";")
    if keys_FR[0]:
        for key_FR in keys_FR:
            if key_FR.strip():
                g.add((dataset, dcat.keyword, Literal(key_FR.strip(), lang="fr")))
            else: pass
    else: print("opendata.swiss french keywords were not provided")  
    keys_IT = ODS_data.get("keywords_IT").split(";")
    if keys_IT[0]:
        for key_IT in keys_IT:
            if key_IT.strip():
                g.add((dataset, dcat.keyword, Literal(key_IT.strip(), lang="it")))
            else: pass    
    else: print("opendata.swiss italian keywords were not provided")  
    keys_EN = ODS_data.get("keywords_EN").split(";")
    if keys_EN[0]:
        for key_EN in keys_EN:
            if key_EN.strip():
                g.add((dataset, dcat.keyword, Literal(key_EN.strip(), lang="en")))
            else: pass
    else: print("opendata.swiss english keywords were not provided")
    
    langs = ODS_data.get("language").split(";")
    if langs[0]:
        for language in langs:
            if language:
                language = language.strip()
                g.add((dataset, dct.language, Literal(language, lang=language)))
            else: pass 
    else: print("opendata.swiss languages missing")

    page = ODS_data.get("landing page")
    if not page: print("opendata.swiss landing page missing")
    elif "https://" in page:
        g.add((dataset, dcat.landingPage, URIRef(page)))
    else: print("opendata.swiss landing page not formatted correctly")

    docu = ODS_data.get("documentation").split(";")
    if docu[0]:
        for doc in docu:
            if not doc: pass
            else: 
                g.add((dataset, URIRef("http://xmlns.com/foaf/0.1/page"), Literal(doc.strip(), datatype="http://xmlns.com/foaf/0.1/Document")))
    else: 
        print("Optional: no opendata.swiss documentation pages provided")
    
    rel_res = ODS_data.get("related resource").split(";")
    if rel_res[0]:
        for res in rel_res:
            if not res: pass
            elif "https://" in res:
                g.add((dataset, dct.relation, Literal(res.strip(), datatype="http://www.w3.org/2000/01/rdf-schema#Resource")))
            else: print("Optional: opendata.swiss related resource not formatted correctly")
    else: print("Optional: no opendata.swiss related resource provided")
    
    qual_rel = ODS_data.get("qualified relation").split(";")
    if qual_rel[0]:
        for rel in qual_rel:
            if not rel: pass
            elif "https://" in rel:
                rel_BN = BNode()
                g.add((dataset, dcat.qualifiedRelation, rel_BN))
                g.add((rel_BN, RDF.type, dcat.Relationship))
                g.add((rel_BN, dcat.hadRole, schema.sameAs))
                g.add((rel_BN, dct.relation, Literal(rel.strip(), datatype="http://www.w3.org/2000/01/rdf-schema#Resource")))
            else: print("Optional: opendata.swiss qualified relation not formatted correctly")
    else: print("Optional: no opendata.swiss qualified relation provided")
    
    temp_start = ODS_data.get("temporal start").split(";")
    if temp_start[0]:
        for tem_sta in temp_start:
            if not tem_sta: pass
            try: 
                datetime.strptime(str(tem_sta).strip(), date_format)
                g.add((dataset, schema.dateCreated, Literal(tem_sta, datatype=URIRef('http://www.w3.org/2001/XMLSchema#date'))))
            except(ValueError):
                print("Optional: opendata.swiss temporal start not formatted correctly")
    else: print("Optional: no opendata.swiss temporal start provided")
    temp_end = ODS_data.get("temporal end").split(";")
    if temp_end[0]:
        for tem_end in temp_end:
            if not tem_end: pass
            try: 
                datetime.strptime(str(tem_end).strip(), date_format)
                g.add((dataset, schema.dateCreated, Literal(tem_end, datatype=URIRef('http://www.w3.org/2001/XMLSchema#date'))))
            except(ValueError):
                print("Optional: opendata.swiss temporal end not formatted correctly")
    else: print("Optional: no opendata.swiss temporal end provided")
    
    spatial = ODS_data.get("spatial")
    if spatial:
        g.add((dataset, dct.spatial, Literal(spatial, lang = "en")))
    else: print("Optional: opendata.swiss spatial information missing")

        
        
        
# Dimensions:

g.add((dataset, cube.observationConstraint, dataset+"/shape"))
#g.add((shape, shacl.property))
 
shape=dataset+"/shape"

g.add((shape, RDF.type, cube.Constraint))
g.add((shape, RDF.type, shacl.NodeShape))
g.add((shape, shacl.closed, Literal('true', datatype=URIRef(xsd.boolean))))

#Dimension 1

Bnode1 =BNode()
g.add((shape, shacl.property, Bnode1))

#One dimension is the observed by <an office> in every data point. Questionable yes, but standard.



g.add((Bnode1, shacl.nodeKind,shacl.IRI ))
g.add((Bnode1, URIRef("http://www.w3.org/ns/shacl#in"),cube.Observation )) #sh.in does not work. Syntax error, don't know why
g.add((Bnode1, schema.name,Literal("Beobachtet von", lang="de") ))
g.add((Bnode1, schema.name,Literal("Observed by", lang="en") ))
g.add((Bnode1, schema.name,Literal("Observé par", lang="fr") ))
g.add((Bnode1, schema.name,Literal("Osservato da", lang="it") ))

#g.add((Bnode1, schema.name, ))
g.add((Bnode1, qudt.scaleType , qudt.NominalScale ))
g.add((Bnode1, RDF.type, cube.KeyDimension))


#All Dimensions



num_dim=input_data.get("NumDimensions")  #generate every dimension 
        
d = {} #create all the blank nodes for all dimension
for x in range(1, num_dim+1):
    d["Bnode{0}".format(x)] = BNode()        

for dim in range(1,num_dim+1):
    print(input_data.get(("dimname_de"+str(dim))))
    print(d[("Bnode"+str(dim))])
    

#for dim in range(1,num_dim+1):
#    if input_data.get(("dimname_de"+str(dim))) :
 #       g.add((d[("Bnode"+str(dim))], schema.name, Literal( input_data.get(("dimname_de"+str(dim))), lang="de")))

    
for dim in range(1,num_dim+1): #todo: second for loop for languages + convert all to the "if inputdata.get..."
    g.add((shape, shacl.property, d[("Bnode"+str(dim))]))
    g.add((d[("Bnode"+str(dim))], schema.name, Literal( input_data.get(("dimname_de"+str(dim))), lang="de")))
    g.add((d[("Bnode"+str(dim))], schema.description, Literal( input_data.get(("dimdesscription_de"+str(dim))), lang="de")))
    g.add((d[("Bnode"+str(dim))], RDF.type, URIRef( input_data.get(("dimtype"+str(dim))))))
    g.add((d[("Bnode"+str(dim))], qudt.scaleType, URIRef( input_data.get(("scaletype"+str(dim))))))
    if input_data.get(("unit"+str(dim))) :
        g.add((d[("Bnode"+str(dim))], qudt.unit, URIRef(input_data.get(("unit"+str(dim)))))) 
    if input_data.get(("datatype"+str(dim))) :
        g.add((d[("Bnode"+str(dim))], qudt.datatype, URIRef(input_data.get(("datatype"+str(dim))))))  
    g.add((d[("Bnode"+str(dim))], shacl.nodeKind, Literal( input_data.get(("nodekind"+str(dim))))))
    #g.add((d[("Bnode"+str(dim))], shacl.path, Pathi ))
    
#to do: at path of observation
Path1= URIRef(dataset_URL+"Jahr")
Path2= URIRef(dataset_URL+"Region")
Path3= URIRef(dataset_URL+"AnzahlgefundeneKleeblätter")
Path4= URIRef(dataset_URL+"AnzahlLottogewinne")


# Observations

df= pd.read_csv("C:\\Users\\claudio\\Documents\\Projekte\\python lottogewinne\\kleeblätter_lottogewinne.csv")

#add observations to datasetdescription 
g.add((dataset, cube.observationSet, dataset+"/observation/")) 
observation =dataset+"/Observation/"

g.add((observation, RDF.type, cube.ObservationSet)) 

#generate the "rows" i.e. dataset_URL + the key-dimensions


key1=df["Jahr"]     #to do: fix this workaround for needing manually insert the right columnnames and automatcally generate as many keys are there are dimension in the yaml
key2=df["Region"]
key3=df["Anzahl gefundener Kleeblätter"]
key4=df["Anzahl Lottogewinne"]



for i in range(0,len(df)):
    g.add((observation, cube.observation, URIRef(dataset_URL+str(key1[i])+"/" +  str(key2[i])))) #Connect all constructed "rows" to graph
    g.add((URIRef(dataset_URL+str(key1[i])+"/" +  str(key2[i])), RDF.type, cube.Observation))
    g.add((URIRef(dataset_URL+str(key1[i])+"/" +  str(key2[i])), cube.observedBy, Literal("Mein Amt"))) #todo amt richtig ausfüllen
    g.add((URIRef(dataset_URL+str(key1[i])+"/" +  str(key2[i])), Path1 , Literal(key1[i])))
    g.add((URIRef(dataset_URL+str(key1[i])+"/" +  str(key2[i])), Path2 , Literal(key2[i])))
    g.add((URIRef(dataset_URL+str(key1[i])+"/" +  str(key2[i])), Path3 , Literal(key3[i])))
    g.add((URIRef(dataset_URL+str(key1[i])+"/" +  str(key2[i])), Path4 , Literal(key4[i])))
    
    
    #print(dataset_URL+str(key1[i])+"/" +  str(key2[i]))



    
if name_DE: filename= name_DE.replace(" ", "_").replace(":","")+".ttl"
elif name_FR: filename= name_FR.replace(" ", "_").replace(":","")+".ttl"
elif name_IT: filename= name_IT.replace(" ", "_").replace(":","")+".ttl"
else: filename= name_EN.replace(" ", "_").replace(":","")+".ttl"
g.serialize(destination=filename)


Missing French dataset name
Missing Italian dataset name
Missing English dataset name
Missing French dataset description
Missing Italian dataset description
Missing English dataset description
Missing source modification date
Visualize work example link is not correctly formatted
SPARQL work example link is missing
dataset creator not formatted correctly
dataset contributor not formatted correctly
dataset publisher not formatted correctly
Optional: no next modification date provided
is it a draft or published? creativeworkstatus wrong
opendata.swiss creator was not formatted correctly
opendata.swiss license missing
opendata.swiss accrual periodicity missing
opendata.swiss theme was not provided
opendata.swiss german keywords were not provided
opendata.swiss french keywords were not provided
opendata.swiss italian keywords were not provided
opendata.swiss english keywords were not provided
opendata.swiss languages missing
opendata.swiss landing page missing
Optional: no opendata.swiss d

<Graph identifier=Nddbb51d49d554c0488c4dae05ecd1807 (<class 'rdflib.graph.Graph'>)>

In [59]:
imname_de1: "Jahr"

dimanme_fr1:
dimdescription_de1: "Loremipsum"
dimdescription_fr1:
dimtype1: "https://cube.link/KeyDimension" #auch https://cube.link/MeasureDimension or empty
scaletype1: "http://qudt.org/schema/qudt/IntervalScale" #to do find the other scale types, qudt:NominalScale
unit1: "http://qudt.org/vocab/unit/YR_Common" #to do: show how to find all available on lindas (in a reasonable way)
datatype1: "http://www.w3.org/2001/XMLSchema#gYear"
#sh.max1
#sh.min1
nodeKind1: "Literal" # rdfs:comment "The class of all node kinds, including sh:BlankNode, sh:IRI, sh:Literal or the combinations of these: sh:BlankNodeOrIRI, sh:BlankNodeOrLiteral, sh:IRIOrLiteral."@en ;
path1: # URI of the Dimensionproperty in an observation. For example Year in https://energy.ld.admin.ch/sfoe/bfe_ogd84_einmalverguetung_fuer_photovoltaikanlagen/9/observation/2014/AG
dataKind1: "Year"  



1
2
3
4


In [116]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import RDF
from dateutil.parser import parse
import yaml
from datetime import datetime
import pandas as pd

#instantiate namespace
schema = Namespace("http://schema.org/")
dcat = Namespace("http://www.w3.org/ns/dcat#")
dct = Namespace("http://purl.org/dc/terms/")
void= Namespace("http://rdfs.org/ns/void#")
cube= Namespace("https://cube.link/")
shacl= Namespace("http://www.w3.org/ns/shacl#")
xsd= Namespace("http://www.w3.org/2001/XMLSchema#")
qudt= Namespace("http://qudt.org/schema/qudt/")

#parse input
with open("C:\\Users\\claudio\\Documents\\Projekte\\python lottogewinne\\input_Form.yml", "rt", encoding='utf8') as yml_input:
    input_data = yaml.load(yml_input, yaml.Loader)

df= pd.read_csv("C:\\Users\\claudio\\Documents\\Projekte\\python lottogewinne\\kleeblätter_lottogewinne.csv")

print(df)

    

     Jahr       Region  Anzahl gefundener Kleeblätter  Anzahl Lottogewinne  \
0    2017            1                            267                 30.7   
1    2017            2                              9                  4.9   
2    2017            3                             33                  7.3   
3    2017            4                            277                 31.7   
4    2017            5                            185                 22.5   
..    ...          ...                            ...                  ...   
157  2022           23                           2151                219.1   
158  2022           24                           1277                131.7   
159  2022           25                            104                 14.4   
160  2022           26                           3560                360.0   
161  2022  Switzerland                          28533               2857.3   

     Kleeblatt Hotspot ID  
0                       1  
1      

In [10]:



d


from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import RDF
from dateutil.parser import parse
import yaml
from datetime import datetime
import sys

sys.path

['C:\\Users\\claudio\\Documents\\Projekte\\python lottogewinne',
 'C:\\Users\\claudio\\anaconda3\\python310.zip',
 'C:\\Users\\claudio\\anaconda3\\DLLs',
 'C:\\Users\\claudio\\anaconda3\\lib',
 'C:\\Users\\claudio\\anaconda3',
 '',
 'C:\\Users\\claudio\\anaconda3\\lib\\site-packages',
 'C:\\Users\\claudio\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\claudio\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\claudio\\anaconda3\\lib\\site-packages\\Pythonwin']

In [ ]:

g.add((Bnode1, schema.name,Literal("Beobachtet von", lang="de") ))
g.add((Bnode1, schema.name,Literal("Observed by", lang="en") ))
g.add((Bnode1, schema.name,Literal("Observé par", lang="fr") ))
g.add((Bnode1, schema.name,Literal("Osservato da", lang="it") ))
g.add((Bnode1, schema.name, ))
g.add((Bnode1, qudt.scaleType,NominalScale  .))
g.add((Bnode1, RDF.Type, cube.KeyDimension))


<https://environment.ld.admin.ch/foen/template_pipeline/1/shape/> a sh:NodeShape, cube:Constraint ;

    sh:closed true ;

    sh:property [ schema:name "Beobachtet von"@de,

                "Observed by"@en,

                "Observé par"@fr,

                "Osservato da"@it ;

            qudt:scaleType qudt:NominalScale ;

            sh:in ( <https://environment.ld.admin.ch/> ) ;

            sh:nodeKind sh:IRI ;

            sh:path cube:observedBy ],

               [ a cube:KeyDimension ;

            schema:name "Warnregion"@de,

                "Warning region"@en,

                "Région d’alerte"@fr,

                "Regione di pericolo"@it ;

            qudt:scaleType qudt:NominalScale ;

            sh:nodeKind sh:IRI ;

           sh:maxCount 1 ;

            sh:minCount 1 ;

            sh:in (REGIONSLISTTOREPLACE) ;

            meta:dataKind [a <http://schema.org/GeoShape>] ;

            sh:path <https://environment.ld.admin.ch/foen/template_pipeline/canton> ],

        [ a cube:MeasureDimension ;

            schema:name "Gefahrenstufe"@de,

                "Danger ratings"@en,

                "Degré de danger"@fr,

                "Grado di pericolo"@it ;

            qudt:scaleType qudt:OrdinalScale ;

                       sh:nodeKind sh:IRI ;

            sh:maxCount 1 ;

            sh:minCount 1 ;

            sh:in (LEVELSLISTTOREPLACE) ;

            sh:path <https://environment.ld.admin.ch/foen/template_pipeline/level> ] ,

        [ schema:name "Aktiv seit"@de,

                "Active since"@en,

                "Actif depuis"@fr,

                "Attivo dal"@it ;

            qudt:scaleType qudt:IntervalScale ;

            sh:nodeKind sh:Literal ;

            sh:path schema:validFrom ]



<http://www.w3.org/ns/shacl#path> <https://health.ld.admin.ch/foph/covid19/dimension/date> .
<http://www.w3.org/ns/shacl#nodeKind> <http://www.w3.org/ns/shacl#Literal> .
<http://www.w3.org/ns/shacl#datatype> <http://www.w3.org/2001/XMLSchema#date> .
# <http://www.w3.org/ns/shacl#minCount> "1"^^<http://www.w3.org/2001/XMLSchema#integer> .
#<http://www.w3.org/ns/shacl#maxCount> "1"^^<http://www.w3.org/2001/XMLSchema#integer> .
# <http://www.w3.org/ns/shacl#minInclusive> "2020-12-21"^^<http://www.w3.org/2001/XMLSchema#date> .
#<http://www.w3.org/ns/shacl#maxInclusive> "2023-07-03"^^<http://www.w3.org/2001/XMLSchema#date> .
 
 <https://cube.link/meta/dataKind> _:genid2dd5a88957033b4b0d8ada88adbe8ccae72d1b110 .
